---
kernelspec:
    name: python3
    display_name: python3
---

# Retrieving ASFV Genome Data

:::{admonition} Objective
Fetch all available ASFV genome assemblies from the NCBI database.
:::

```{code-cell} python
:tags: remove-input
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
```

## Querying the NCBI Assembly Database

[](#fetch-assemblies) retrieves all ASFV assembled genomes hosted at the NCBI Assembly database. The `efetch` command queries the NCBI nucleotide database (nuccore) using the given query string. The results are then piped into `elink` to convert the returned UIDs into assembly IDs prefixed with "GCA". Metadata for each assembly ID is then extracted using `esummary` and parsed with the supplemetary `xtract` tools also provided by `edirect`. The final output is a list of assembly IDs stored as a newline-delimited text file (`accessions.txt`).

:::{code-block} bash
:label: fetch-assemblies
:caption: Retrieve ASFV assemblies from the Philippines.

OUT=assets/flatfiles/accessions.txt
esearch -db nuccore -query "African swine fever virus complete genome AND Philippines" \
    | elink -target assembly \
    | esummary \
    | xtract -pattern DocumentSummary -element AssemblyAccession > ${OUT}
:::

Additionally, the assembly IDs of representative genomes of other ASFV genotypes as categorized by @doi:10.3390/v15112246 were also retrieved by running [](#fetch-other-assemblies). The returned IDs were appended to the `accessions.txt`.

:::{code-block} bash
:label: fetch-other-assemblies
:caption: Retrieve ASFV assemblies representative of other genotypes.

OUT=assets/flatfiles/accessions.txt
ACC=assets/flatfiles/new_genotype_accessions.txt
cat ${ACC} \
    | elink -db nuccore -target assembly \
    | esummary \
    | xtract -pattern DocumentSummary -element AssemblyAccession >> ${OUT}
:::

The output file should contain a list of  **29** assembly identifiers, 20 from the Philippines and 9 from @doi:10.3390/v15112246. We could confirm this by displaying the contents of `accessions.txt`.

:::{code-cell} python
:tags: remove-input
:label: count-accessions

!bat assets/flatfiles/accessions.txt
:::

## Filtering for Complete Assemblies

Only complete assemblies will be used to construct the pangenome graph. To do this, we include the "AssemblyStatus" field when parsing using `xtract` and subsequently filtering the results with `grep`.

```{code-block} bash
:label: filter-complete-genomes
:caption: Filter for accessions with complete genome assemblies.

OUT=./assets/flatfiles/complete_assemblies.csv
echo "accession,assembly_id,status,date_published" > ${OUT}
grep "Complete Genome" ./assets/flatfiles/assemblies.csv \
    | sed -e "s/complete genome,//g" >> ${OUT}
```

```{code-block} bash
:label: count-genomes
:caption: Count the number of assembled genome per year and visualize.

cat ./assets/flatfiles/assemblies.csv \
    | awk -F , '{print $4}' | awk -F / '{print $1}' \
    | sort -n -r | uniq -c | head -n -1 \
    | awk '{$1=$1;print}' | tr " " "," > ./assets/flatfiles/counts.csv
```

```{code-cell} python
:tags: remove-input
:label: genome-counts

genome_counts = pd.read_csv("./assets/flatfiles/counts.csv", header=None, names=["count", "year"]).sort_values(by="year", ascending=True)

fig, ax = plt.subplots(figsize=(10,6))
ax.bar(x=genome_counts["year"], height=genome_counts["count"])
ax.set_xlabel("Year")
ax.set_ylabel("Genome count")
ax.set_title("No. of Assembled ASFV Genomes Per Year") 

for rect in ax.patches:
    y_value = rect.get_height()
    x_value = rect.get_x() + rect.get_width() / 2
    space = 1
    label = "{:.0f}".format(y_value)
    ax.annotate(label, (x_value, y_value), xytext=(0, space), textcoords="offset points", ha='center', va='bottom')

plt.show()
```

ASFV assemblies can be aggregated by count across different years by running [](#count-genomes). As of writing there are a total of {eval}`genome_counts['count'].sum()` available ASFV genomes, 347 of which are complete assemblies.

## Downloading Genomes

`datasets` will be used to request all assemblies from the NCBI server as seen in [](#download-all-genomes). The list of accessions identifiers will be passed as argument to the `--inputfile` flag.

```{code-block} bash
:label: download-all-genomes
:caption: Request all assemblies from the NCBI server using `datasets`.

# Create an directory for genome assemblies
mkdir -p assets/genomes

# Download all assemblies from NCBI
datasets download genome accession --inputfile assets/flatfiles/accessions.txt \
    --include genome --filename assets/genomes.zip'
``` 

All downloaded genomes are stored in the zipped parent directory (`assets/genomes.zip`). This corresponds to exactly 29 FASTA files. Some assemblies have multiple versions as evident by the same accession having different suffixes (e.g., `GCA_003815435.1`, `GCA_003815435.2`, `GCA_003815435.1`). Only the most recent version was retained for analysis.

All directories are decompressed and sequnces are merged into a single, large FASTA file by running [](#concat-all-fasta).

```{code-block} bash
:label: concat-all-fasta
:caption: Merge all FASTA files using `cat`.

# Unzip downloaded directory
unzip assets/genomes.zip -d assets/genomes

# Merge all FASTA files into a single file
cat assets/genomes/ncbi_datasets/data/**/*.fna > assets/genomes.fasta
```

As a sanity check, confirm that there are 29 header lines in `genomes.fasta`:

In [2]:
!cat ../assets/genomes.fasta | grep ">" | wc -l

29
